In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp pod.client

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Pod Client

In [ ]:
# export
from integrators.itembase import Edge, ItemBase
from integrators.schema import *
from integrators.imports import *
import hashlib

In [ ]:
# export
API_URL = "http://localhost:3030/v2"

In [ ]:
# export

class PodClient:

    def __init__(self, url=API_URL, database_key=None, owner_key=None):
        self.url = url
        self.base_url = f"{url}/{owner_key}"
        self.test_connection(verbose=False)
        self.database_key=database_key
        self.owner_key=owner_key
        
    # PRIMITIVE FUNCTIONS
    def version(self):
        result = requests.get(f'{self.url}/version',
                      verify=False)
        print(result.content)
    
    def get_item(self, uid):
        wrapped_item = {
            'databaseKey': self.database_key,
            'payload': uid,
        }
        result = requests.post(f'{self.base_url}/get_item',
                      json=wrapped_item,
                      verify=False)

        if result.ok:
            # TODO: get_item actually returns a list
            return result.json()
        else:
            # TODO: better exception handling
            raise Exception(result.status_code, result.text)

    def get_all_items(self):
        NotImplementedError()
    
    def create_item(self, item, item_type=None):
        # TODO: unify with create_item once https/http difference is resolved
        if item_type != None:
            item['_type'] = item_type

        wrapped_item = {
            'databaseKey': self.database_key,
            'payload': item,
        }
        result = requests.post(f'{self.base_url}/create_item',
                      json=wrapped_item,
                      verify=False)

        if result.ok:
            return result.json()
        else:
            # TODO: better exception handling
            raise Exception(result.status_code, result.text)
            
    def update_item(self, item):
        wrapped_item = {"databaseKey": self.database_key,
                        "payload": item}

        try:
            result = requests.post(f"{self.base_url}/update_item",
                                  json=wrapped_item)
            if result.status_code != 200:
                print(result, result.content)
        except requests.exceptions.RequestException as e:
            print(e)
        
    def delete_item(self, item):
        NotImplementedError()
        
    def bulk_action(self, create_items, update_items, delete_items, create_edges, delete_edges):
        edges_data = {"databaseKey": self.database_key, 
                      "payload": {
                          "createItems": create_items, 
                          "updateItems": update_items, 
                          "deleteItems": delete_items, 
                          "createEdges": create_edges,
                          "deleteEdges": delete_edges
                      }}
        
        try:
            result = requests.post(f"{self.base_url}/bulk_action",
                                   json=edges_data,
                                   verify=False)
            if result.status_code != 200:
                if "UNIQUE constraint failed" in str(result.content):
                    print(result.status_code, "Edge already exists")
                else:
                    print(result, result.content)
                return False
            else:
                return True
        except requests.exceptions.RequestException as e:
            print(e)
            return False
        
    def search_by_fields(self, fields_data):
        body = {"databaseKey": self.database_key,
                "payload": fields_data}
        try:
            result = requests.post(f"{self.base_url}/search_by_fields",
                                   json=body)
            json =  result.json()
            return [self.item_from_json(item) for item in json]
        except requests.exceptions.RequestException as e:
            return None
        
    def get_items_with_edges(self, item_uids):
        body = {"databaseKey": self.database_key,
                "payload": item_uids,}
        
        try:
            result = requests.post(f"{self.base_url}/get_items_with_edges",
                                    json=body,
                                    verify=False)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                print(result.json())
                json = result.json()[0]
                res =  self.item_from_json(json)
                return res

        except requests.exceptions.RequestException as e:
            print(e)
            return None
    
    def upload_file(self, file, hash):
        result = requests.post(f'{self.base_url}/upload_file/{self.database_key}/{hash}',
                               file,
                               verify=False)
        print(result.status_code)

        if result.ok:
            return True
        else:
            if result.status_code == 409:
                return False
            raise Exception(result.status_code, result.text)
            
    def get_file(self, hash):
        wrapped_item = {
            'databaseKey': self.database_key,
            'payload': {
                'sha256': hash,
            }
        }

        result = requests.post(f'{self.base_url}/get_file',
                               json=wrapped_item,
                               verify=False)

        if result.ok:
            return result.content
        else:
            raise Exception(result.status_code, result.text)
            
    # END OF PRIMITIVE FUNCTIONS
    
    def test_connection(self, verbose=True):
        # TODO: change this function, remove get-request
        try:
            res = requests.get(self.url, verify=False)
            if verbose: print("Succesfully connected to pod")
            return True
        except requests.exceptions.RequestException as e:
            print("Could no connect to backend")
            return False
    
    def create(self, node):
        self.create_item(node.to_dict())
        
        # todo: ItemBase.add_to_db(node) if succesful
    
    def create_edges(self, edges):
        """Create edges between nodes, edges should be of format [{"_type": "friend", "_source": 1, "_target": 2}]"""
        edges_data = []
        for e in edges:
            src, target = e.source.uid, e.target.uid
            data = {"_source": src, "_target": target, "_type": e._type}
            if e.label is not None: data[LABEL] = e.label
            if e.sequence is not None: data[SEQUENCE] = e.sequence

            edges_data.append(data)
            
            if e.reverse:
                data2 = copy(data)
                data2["_source"] = target
                data2["_target"] = src
                data2["_type"] = "~" + data2["_type"]
                edges_data.append(data2)

        self.bulk_action([], [], [], edges_data, [])
        
    def create_edge(self, edge):
        return self.create_edges([edge])
    
    def get(self, uid, expanded=True):
        if not expanded:
            return self.get_item(uid)
        else:
            return self.get_items_with_edges([uid])

    def item_from_json(self, json):
        indexer_class = json.get("indexerClass", None)
        constructor = get_constructor(json["_type"], indexer_class)
        new_item = constructor.from_json(json)
        existing = ItemBase.global_db.get(new_item.uid)
        # TODO: cleanup
        if existing is not None:
            if not existing.is_expanded() and new_item.is_expanded():
                existing.edges = new_item.edges
            return existing
        else:
            return item

    def get_properties(self, expanded):
        properties = copy(expanded)
        if ALL_EDGES in properties: del properties[ALL_EDGES]
        return properties

    def run_importer(self, uid, servicePayload):

        body = dict()
        body["databaseKey"] = servicePayload["databaseKey"]
        body["payload"] = {"uid": uid, "servicePayload": servicePayload}

        print(body)

        try:
            res = requests.post(f"{self.base_url}/run_importer", json=body)
            # res = requests.post(self.url)
            if res.status_code != 200:
                print(f"Failed to start importer on {url}:\n{res.status_code}: {res.text}")
            else:
                print("Starting importer")
        except requests.exceptions.RequestException as e:
            print("Error with calling importer {e}")
            
    def upload_and_create_file(self, file):
        hash = hashlib.sha256(file).hexdigest()
        
        # TODO: check if creation was succesful
        file_item = {
            'sha256': hash
        }
        self.create_item(file_item, item_type='File')
#         file_item = Item()
#         file_item.sha256 = hash
#         self.create(file_item)

        self.upload_file(file, hash)
        
        return file_item

In [ ]:
from unittest import mock
import unittest

def MockRequest(status_code=200, json_object=None, text=''):
    response = requests.Response()
    response.status_code = status_code
    if json_object != None:
        response._content = bytes(json.dumps(json_object), 'utf-8')
    else:
        response._content = bytes(text, 'utf-8')
    return response

# @mock.patch('requests.post', mock.Mock(side_effect = mocked_requests_post))
# # @mock.patch('requests.post', MagicMock(name='method'))
# def test_fetch():
#     pod_client.create_item({"aa": "bb"})
    
# test_fetch()

url = 'https://localhost:3030/v2'
database_key = "0" * 64
owner_key = "1" * 64
pod_client = PodClient(url=url, database_key=database_key, owner_key=owner_key)

class PodPrimitivesTestCase(unittest.TestCase):
    @mock.patch('requests.post', mock.Mock())
    def test_create_item(self):
        #with mock.patch('requests.post') as request_mock:
        pod_client.create_item({"aa": "bb"})

        expected_json = {'databaseKey': database_key, 
                         'payload': {'aa': 'bb'}}
        requests.post.assert_called_once_with(f'{url}/{owner_key}/create_item', json=expected_json, verify=False)

    @mock.patch('requests.post', mock.Mock(return_value = MockRequest()))
    def test_upload_file(self):
        file = b'aa'
        file_hash = 'bb'
        result = pod_client.upload_file(file, file_hash)
        assert result == True
        requests.post.assert_called_once_with(f'{url}/{owner_key}/upload_file/{database_key}/{file_hash}', file, verify=False)

    # Test the file upload primitive (file already exists)
    @mock.patch('requests.post', mock.Mock())
    def test_upload_file_existing(self):
        file = b'aa'
        file_hash = 'bb'
        requests.post.return_value = MockRequest(status_code=409, json_object={'a':'b'})
        result = pod_client.upload_file(file, file_hash)
        assert result == False
        requests.post.assert_called_once_with(f'{url}/{owner_key}/upload_file/{database_key}/{file_hash}', file, verify=False)

    # Test the file upload primitive (General error)
    @mock.patch('requests.post', mock.Mock())
    def test_upload_file_exception(self):
        file = b'aa'
        file_hash = 'bb'
        requests.post.return_value = MockRequest(status_code=408)

        self.assertRaises(Exception, pod_client.upload_file, file, file_hash)
        requests.post.assert_called_once_with(f'{url}/{owner_key}/upload_file/{database_key}/{file_hash}', file, verify=False)
        
    # Test the file downloading primitive
    @mock.patch('requests.post', mock.Mock())
    def test_get_file(self):
        file = b'aa'
        file_hash = 'bb'
        requests.post.return_value = MockRequest()

        result = pod_client.get_file(file_hash)
        # TODO: test that the result is the file
        expected_json = {'databaseKey': database_key, 
                         'payload': {'sha256':file_hash}}
        requests.post.assert_called_once_with(f'{url}/{owner_key}/get_file', json=expected_json, verify=False)

        

suite = unittest.TestLoader().loadTestsFromTestCase(PodPrimitivesTestCase)
result = unittest.TextTestRunner(verbosity=3).run(suite)

print(result)

/Users/user/Desktop/Work/Polis/Repositories/pyintegrators/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
test_create_item (__main__.PodPrimitivesTestCase) ... ok
test_get_file (__main__.PodPrimitivesTestCase) ... ok
test_upload_file (__main__.PodPrimitivesTestCase) ... ok
test_upload_file_exception (__main__.PodPrimitivesTestCase) ... ok
test_upload_file_existing (__main__.PodPrimitivesTestCase) ... 

200
408
409
<unittest.runner.TextTestResult run=5 errors=0 failures=0>


ok

----------------------------------------------------------------------
Ran 5 tests in 0.007s

OK


In [ ]:
from unittest.mock import MagicMock

# This is the pod_client, but mocking all http-using functions
def create_mocked_pod_client():
    pod_client = PodClient(url='https://localhost:3030/v2', database_key="0" * 64, owner_key="1" * 64)
    
    pod_client.get_item = MagicMock(name='method')
    # get_all_items
    
    pod_client.create_item = MagicMock(name='method')
    pod_client.update_item = MagicMock(name='method')
    pod_client.bulk_action = MagicMock(name='method')
    pod_client.delete_item = MagicMock(name='method')
    
    # search_by_fields
    pod_client.get_items_with_edges = MagicMock(name='method')
    
    pod_client.upload_file = MagicMock(name='method')
    pod_client.get_file = MagicMock(name='method')
    
    return pod_client

/Users/user/Desktop/Work/Polis/Repositories/pyintegrators/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [ ]:
# Test the creation of edges

pod_client = create_mocked_pod_client()

alice = Person.from_data(firstName="Alice", uid=1)
bob = Person.from_data(firstName="Bob", uid=2)

# Test both normal and bidirectional edges
edge1 = Edge(alice, bob, "friend", reverse=True)
edge2 = Edge(alice, bob, "secretly_admires", reverse=False)
pod_client.create_edges([edge1, edge2])

expected_edges = [{'_source': 1, '_target': 2, '_type': 'friend'},
                  {'_source': 2, '_target': 1, '_type': '~friend'},
                  {'_source': 1, '_target': 2, '_type': 'secretly_admires'},]
pod_client.bulk_action.assert_called_once_with([], [], [], expected_edges, [])

/Users/user/Desktop/Work/Polis/Repositories/pyintegrators/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [ ]:
# Test the retrievel of items, both with and without expanded

pod_client = create_mocked_pod_client()
pod_client.get(1, expanded=True)
pod_client.get_items_with_edges.assert_called_once_with([1])

pod_client = create_mocked_pod_client()
pod_client.get(1, expanded=False)
pod_client.get_item.assert_called_once_with(1)

/Users/user/Desktop/Work/Polis/Repositories/pyintegrators/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/user/Desktop/Work/Polis/Repositories/pyintegrators/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [ ]:
# Test the create function

pod_client = create_mocked_pod_client()

alice = Person.from_data(firstName="Alice")
pod_client.create(alice)
pod_client.create_item.assert_called_once_with(alice.to_dict())

/Users/user/Desktop/Work/Polis/Repositories/pyintegrators/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:988: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [ ]:
# Test the uploading of a file, and the related helper function

pod_client = create_mocked_pod_client()

file = b'aa'
file_hash = hashlib.sha256(file).hexdigest()
expected_file_item = {'sha256': file_hash}

pod_client.upload_and_create_file(file)
pod_client.create_item.assert_called_once_with(expected_file_item, item_type='File')
pod_client.upload_file.assert_called_once_with(file, file_hash)

We communicate with the pod with the PodClient. The PodClient requires us to provide a [database key](https://gitlab.memri.io/memri/pod/-/blob/dev/docs/HTTP_API.md#user-content-api-authentication-credentials) and an [owner key](https://gitlab.memri.io/memri/pod/-/blob/dev/docs/HTTP_API.md#user-content-api-authentication-credentials). You don't have to worry about these keys: when you run an Integrator from a memri client, this goes via the pod, which provides these keys for you. For testing purposes, we can make our own keys.

In [ ]:
client = PodClient(database_key="0" * 64, owner_key="1" * 64)
success = client.test_connection()
assert success

Succesfully connected to pod


## Creating Items and Edges

Now that we have access to the pod, we can create items here and upload them to the pod. All items are defined in the memri [schema](https://gitlab.memri.io/memri/schema). When the schema is changed it automatically generates all the class definitions for the different languages used in memri, the python schema file lives in [schema.py](https://gitlab.memri.io/memri/pyintegrators/-/blob/master/integrators/schema.py) in the integrators package. When Initializing an Item, always make sure to use the from_data classmethod to initialize.

In [ ]:
email_item = EmailMessage.from_data(content="example content field")
email_item

EmailMessage (#None)

In [ ]:
success = client.create(email_item)
assert success
email_item

EmailMessage (#917040)

We can connect items using edges. Let's create another item, a person, and connect the email and the person.

In [ ]:
person_item = Person.from_data(firstName="Alice")
item_succes = client.create(person_item)

edge = Edge(person_item, email_item, "author")
edge_succes = client.create_edge(edge)

assert item_succes and edge_succes
edge

Person (#917041) --author-> EmailMessage (#917040)

# Fetching and updating Items

We can use the client to fetch data from the database. This is in particular usefull for indexers, which often use data in the database as input for their models. The simplest form  of querying the database is by querying items in the pod by their uid (unique identifier).

In [ ]:
person_item = Person.from_data(firstName="Alice")
client.create(person_item)
person_from_db = client.get(person_item.uid)
assert person_from_db is not None
assert person_from_db == person_item
person_from_db

Person (#917042)

Appart from creating, we might want to update existing items:

In [ ]:
person_item.lastName = "Awesome"
client.update_item(person_item)

person_from_db = client.get(person_item.uid)
assert person_from_db.lastName == "Awesome"
person_from_db

Person (#917042)

Sometimes, we might not know the uids of the items we want to fetch. We can also search by a certain property. We can use this for instance when we want to query all items from a particular type to perform some indexing on.

In [ ]:
person_item2 = Person.from_data(firstName="Bob")
client.create(person_item2);
all_people = client.search_by_fields({"_type": "Person"})

assert all([isinstance(p, Person) for p in all_people]) and len(all_people) > 0

all_people[:3]

[Person (#32002), Person (#32004), Person (#32005)]

# Export -

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted index.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted pod.client.ipynb.
